In [1]:
import pandas as pd
import numpy as np

# Load data

In [2]:
train_values = pd.read_csv('../data/train_values.csv', index_col='building_id')
train_values.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,0
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,0


In [3]:
train_labels = pd.read_csv('../data/train_labels.csv', index_col='building_id')
train_labels.head()

,damage_grade
building_id,
802906,3
28830,2
94947,3
590882,2
201944,3


# Explore features

In [4]:
# Categorical columns = 
categorical_columns = [c for c in train_values.select_dtypes(include=['object'])]
for c in categorical_columns:
    print(c)

land_surface_condition
foundation_type
roof_type
ground_floor_type
other_floor_type
position
plan_configuration
legal_ownership_status


In [5]:
numerical_columns = list(set(train_values.columns) - set(categorical_columns))
numerical_columns

['count_families',
 'geo_level_2_id',
 'has_superstructure_other',
 'has_superstructure_cement_mortar_stone',
 'has_superstructure_mud_mortar_brick',
 'has_superstructure_mud_mortar_stone',
 'has_secondary_use_other',
 'area_percentage',
 'has_superstructure_adobe_mud',
 'has_superstructure_stone_flag',
 'has_secondary_use_use_police',
 'has_secondary_use_gov_office',
 'geo_level_1_id',
 'has_secondary_use_rental',
 'has_superstructure_timber',
 'has_superstructure_rc_non_engineered',
 'has_secondary_use_health_post',
 'geo_level_3_id',
 'has_secondary_use_industry',
 'has_superstructure_cement_mortar_brick',
 'age',
 'has_superstructure_bamboo',
 'has_secondary_use_hotel',
 'has_secondary_use_institution',
 'has_secondary_use_school',
 'count_floors_pre_eq',
 'has_secondary_use',
 'height_percentage',
 'has_secondary_use_agriculture',
 'has_superstructure_rc_engineered']

In [6]:
train_values.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,0
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,0


### Drop duplicates

In [7]:
duplicate_index_mask = train_values.index.duplicated(keep='first')
train_values = train_values[~duplicate_index_mask]
train_labels = train_labels[~duplicate_index_mask]

### Drop the index column

In [8]:
# Drop building_id (index) from X and y
train_values.reset_index(drop=True, inplace=True)
train_labels.reset_index(drop=True, inplace=True)


## Split the data

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train_values, train_labels, train_size=0.8, test_size=0.2, random_state=0)

All possible values from all categorical columns are present in the train set

## Encode categorical values

In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('ordinal', OrdinalEncoder())
])

# Bundle preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_columns),
        ("numerical", "passthrough", numerical_columns),
    ])

In [11]:
# for preprocessing the data
#from sklearn.preprocessing import StandardScaler

# the model
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=57)


In [12]:
clf = Pipeline(
    steps=[('preprocessor', preprocessor),
           ('model', model)])

clf.fit(X_train, y_train)

/home/adam/miniconda3/envs/dsrPython11/lib/python3.11/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [ ]:
print(f"Model score: {clf.score(X_valid, y_valid)}")

Model score: 0.7186738550680148


# Prediction on validation data

In [ ]:
preds_valid = clf.predict(X_valid)

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_valid, preds_valid, average='micro')

0.7186738550680147

# Prediction on test data

In [ ]:
X_test = pd.read_csv('../data/test_values.csv', index_col='building_id')
# Drop building_id (index)
X_test.reset_index(drop=True, inplace=True)


In [ ]:
preds_test = clf.predict(X_test)

# Results submission

In [ ]:
X_test = pd.read_csv('../data/test_values.csv', index_col='building_id')
# Drop building_id (index)
X_test.reset_index(drop=True, inplace=True)


In [ ]:
submission_format = pd.read_csv('../data/submission_format.csv', index_col='building_id')
my_submission = pd.DataFrame(data=preds_test,
                             columns=submission_format.columns,
                             index=submission_format.index)
my_submission.head()

,damage_grade
building_id,
300051,2
99355,2
890251,2
745817,1
421793,3


In [ ]:
my_submission.to_csv('submission.csv')